**Importing Required Libraries**

In [ ]:
## Importing Required Libraries

import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

**Importing face recognition and mask recognition models**

In [ ]:
##Importing face recognition and mask recognition models

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
model = load_model("mask_recog.h5")

**Function to identify mask**

In [ ]:
## Function to identify mask

def face_mask_detector(frame):
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(gray,
                                        scaleFactor=1.1,
                                        minNeighbors=5,
                                        minSize=(60, 60),
                                        flags=cv2.CASCADE_SCALE_IMAGE)
  faces_list=[]
  preds=[]
  for (x, y, w, h) in faces:
      face_frame = frame[y:y+h,x:x+w]
      face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
      face_frame = cv2.resize(face_frame, (224, 224))
      face_frame = img_to_array(face_frame)
      face_frame = np.expand_dims(face_frame, axis=0)
      face_frame =  preprocess_input(face_frame)
      faces_list.append(face_frame)
      if len(faces_list)>0:
          preds = model.predict(faces_list)
      for pred in preds:
          (mask, withoutMask) = pred
      label = "Mask" if mask > withoutMask else "No Mask"
      color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
      label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
      cv2.putText(frame, label, (x, y- 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

      cv2.rectangle(frame, (x, y), (x + w, y + h),color, 3)
  return frame

**Image with mask**

In [ ]:
## Image with mask

input_image = cv2.imread("imagemask.jpg")
imgResize = cv2.resize(input_image, (400,400))
output = face_mask_detector(imgResize)
cv2_imshow(output)

** Image without mask **

In [ ]:
## Image without mask

input_image = cv2.imread("image.jpg")
imgResize = cv2.resize(input_image, (400, 400))
output = face_mask_detector(imgResize)
cv2_imshow(output)